In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.simplefilter("ignore", UserWarning)
import pandas as pd
import csv
import sys
import array
import xarray as xr
import cmocean.cm as cmocean
def write_with_byte_inversion(f, tab):
    tab2write = array.array('f', tab.T.flatten())
    if sys.byteorder == 'little':
        tab2write.byteswap()
    f.write(tab2write)

dtype=np.float32
#topo_adjust to topo wi
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
from matplotlib.ticker import FuncFormatter
import pathlib
from pathlib import Path
import os
import netCDF4
from netCDF4 import Dataset as nc
import glob
from glob import glob
import glidertools as gt
import gsw

In [ ]:
ds_work = xr.open_dataset('ds_clean.nc')
ds_work

In [ ]:
density_comp = gsw.sigma0(ds_work.salinity, ds_work.temp)


In [ ]:
# grid
depth = np.arange(0,1000,1)
latitude = gt.grid_data(ds_work.dives, ds_work.ctd_depth, ds_work.latitude, bins=depth)
longitude = gt.grid_data(ds_work.dives, ds_work.ctd_depth, ds_work.longitude, bins= depth)
temp = gt.grid_data(ds_work.dives, ds_work.ctd_depth, ds_work.temp, bins= depth)
sal = gt.grid_data(ds_work.dives, ds_work.ctd_depth, ds_work.salinity, bins= depth)
o2 = gt.grid_data(ds_work.dives, ds_work.ctd_depth, ds_work.o2, bins= depth)
p = gt.grid_data(ds_work.dives, ds_work.ctd_depth, ds_work.pressure, bins = depth)
density = gt.grid_data(ds_work.dives, ds_work.ctd_depth, density_comp, bins = depth)
optodetemp = gt.grid_data(ds_work.dives, ds_work.ctd_depth, ds_work.optode_temp, bins=depth)

In [ ]:
plt.plot(sal.sel(dives=26.5), sal.ctd_depth, label= 'orginal data')
#plt.ylim(0,80)
#plt.xlim(12,16.5)
#plt.gca().invert_yaxis()
#plt.plot(sal_f_g.sel(dives=26.5), sal_f_g.ctd_depth, label='filtered data')
#plt.ylim(30,50)
#plt.xlim(4,7)
plt.gca().invert_yaxis()
plt.xlabel('Sal')
plt.ylabel('Depth')
plt.grid()
plt.legend()
plt.ylim(100,0)

In [ ]:
# Calculate the mean longitude for each dive
mean_longitude_per_dive = longitude.mean(dim='ctd_depth')

# Print the mean longitude for each dive
#mean_longitude_per_dive

# Calculate the mean latititude for each dive
mean_latitude_per_dive = latitude.mean(dim='ctd_depth')

# Print the mean longitude for each dive
#mean_latitude_per_dive

In [ ]:
temp

In [ ]:
plt.plot(temp.sel(dives=249.5), temp.ctd_depth)
plt.plot(temp.sel(dives=250), temp.ctd_depth)
plt.xlim(14,18)
plt.ylim(0,200)

In [ ]:
mean_longitude_per_dive

In [ ]:
# creating new dataset

# Extract the data from the DataArray objects
temp_data = temp.values
salinity_data = sal.values
o2_data = o2.values
ctd_pressure_data= p.values
density_data = density.values
optode_temp_data = optodetemp.values
#latitude_data = ds_work.latitude.values
#longitude_data = ds_work.longitude.values


# Get the dimension names for temp, salinity, and o2
temp_dims = temp.dims
salinity_dims = sal.dims
o2_dims = o2.dims
ctd_pressure_dims = p.dims
density_dims = density.dims
optode_temp_dims = optodetemp.dims
#latitude_dims = ds_work.latitude_gsm.dims
#longitude_dims = ds_work.longitude_gsm.dims


# Create the Dataset
ds = xr.Dataset(
    data_vars=dict(
        temp=(temp_dims, temp_data),
        salinity=(salinity_dims, salinity_data),
        o2=(o2_dims, o2_data),
        pressure=(ctd_pressure_dims, ctd_pressure_data),
        density=(density_dims, density_data),
        optode_temp = (optode_temp_dims, optode_temp_data),
        #latitude = (latitude_dims, latitude_data),
        #longitude  = (longitude_dims, longitude_data),
        
    ),
    coords=dict(
        dives=temp.dives,
        depth=temp.ctd_depth,
        longitude=mean_longitude_per_dive,
        latitude=mean_longitude_per_dive,
    ),
    attrs=dict(description="dataset including gridded data and cleaned variables"),
)

# Print the dataset
print(ds)

In [ ]:
ds.to_netcdf('ds_gridded.nc')

In [ ]:
# Convert micromoles per kilogram (μmol/kg) to moles per kilogram (mmol/kg)
oxygen_mol_kg = ds.o2 / 1000

# density of water (in kg/m³)
water_density =  ds.density + 1000

# Convert moles per kilogram (mmol/kg) to millimoles per cubic meter (mmol/m³)
oxygen_mol_per_kg_to_mmol_per_m3 = oxygen_mol_kg * water_density 
oxygen_mol_per_kg_to_mmol_per_m3

In [ ]:
# creating new dataset

# Extract the data from the DataArray objects
temp_data = temp.values
salinity_data = sal.values
o2_data = o2.values
ctd_pressure_data= p.values
density_data = density.values
oxygen_mol_per_kg_to_mmol_per_m3_data = oxygen_mol_per_kg_to_mmol_per_m3.values
#latitude_data = ds_work.latitude.values
#longitude_data = ds_work.longitude.values


# Get the dimension names for temp, salinity, and o2
temp_dims = temp.dims
salinity_dims = sal.dims
o2_dims = o2.dims
ctd_pressure_dims = p.dims
density_dims = density.dims
oxygen_mol_per_kg_to_mmol_per_m3_dims = oxygen_mol_per_kg_to_mmol_per_m3.dims
#latitude_dims = ds_work.latitude_gsm.dims
#longitude_dims = ds_work.longitude_gsm.dims


# Create the Dataset
ds_new = xr.Dataset(
    data_vars=dict(
        temp=(temp_dims, temp_data),
        salinity=(salinity_dims, salinity_data),
        o2=(o2_dims, o2_data),
        pressure=(ctd_pressure_dims, ctd_pressure_data),
        density=(density_dims, density_data),
        oxygen_mol_per_kg_to_mmol_per_m3 = (oxygen_mol_per_kg_to_mmol_per_m3_dims, oxygen_mol_per_kg_to_mmol_per_m3_data),
        #latitude = (latitude_dims, latitude_data),
        #longitude  = (longitude_dims, longitude_data),
        
    ),
    coords=dict(
        dives=temp.dives,
        ctd_depth=temp.ctd_depth,
        longitude=mean_longitude_per_dive,
        latitude=mean_latitude_per_dive,
    ),
    attrs=dict(description="dataset including gridded data and cleaned variables"),
)

# Print the dataset
print(ds_new)

In [ ]:
ds_new

In [ ]:
ox_upcast = xr.open_dataset('ox_upcast_data.nc')
ox_downcast = xr.open_dataset('ds_ctd_final.nc')

In [ ]:
# Extract oxygen data at station 15
#oxygenup_station_15 = ox_upcast.sel(station=15)
# Extract oxygen data at station 15
oxygendown_station_15 = ox_downcast.sel(station=15, cast= 'down')

In [ ]:
oxygendown_station_15

In [ ]:
# Extract longitude and latitude
down_longitude_station_15 = oxygendown_station_15.longitude.values
down_latitude_station_15 = oxygendown_station_15.latitude.values

print("Longitude downtake at Station 15:", down_longitude_station_15)
print("Latitude downtake at Station 15:", down_latitude_station_15)

In [ ]:
# Filter the dataset for the desired longitude and latitude
longitude = down_longitude_station_15
latitude = down_latitude_station_15

# Select the variable 'oxygen_concentration_mmm3' from the dataset
oxygen_concentration = oxygendown_station_15['ox_downcast']

# Extract pressure values for station 15
pressure_levels_station_15 = oxygendown_station_15.pressure.values

# Plot oxygen data against pressure at station 15
plt.figure(figsize=(10, 6))
plt.plot(pressure_levels_station_15, oxygen_concentration, color='red', label='Station 15 (mmol /m3)' )

# Define the proximity threshold (in degrees)
proximity_threshold = 0.006 # Adjust as needed

# Filter the dataset for data points within the proximity threshold
filtered_data = ds_new.where(
    (np.abs(ds_new['longitude'] - longitude) < proximity_threshold) &
    (np.abs(ds_new['latitude'] - latitude) < proximity_threshold),
    drop=True
)

# Plot dissolved oxygen against pressure for filtered data
if len(filtered_data['pressure']) != 0:
    plt.plot(filtered_data['pressure'], filtered_data['oxygen_mol_per_kg_to_mmol_per_m3'], label='Seaglider Data (mmol/m3)', color='blue', linestyle='--')


plt.xlabel('Pressure')
plt.ylabel('Oxygen concentration')
plt.title('Oxygen concentration vs Pressure')
plt.grid(True)
plt.legend()
plt.xlim(0, 1000)
plt.show()

In [ ]:
plt.plot(filtered_data.pressure, filtered_data.temp, label= 'glider temp')
plt.plot(oxygendown_station_15.pressure, oxygendown_station_15.temp, label='ctd temp')
plt.xlim(0, 1000)
plt.xlabel('pressure')
plt.ylabel('temperature')
plt.legend()
plt.grid()


In [ ]:
sliced_data = oxygendown_station_15.isel(pressure=slice(999))


In [ ]:
sliced_data

In [ ]:
filtered_data

In [ ]:
ds2=filtered_data.copy()
ds2=ds2.rename_vars({"pressure":"P"})
ds2=ds2.rename_dims({"ctd_depth":"pressure"})
ds2["ctd_depth"]=ds2.ctd_depth-0.5
ds2

In [ ]:
filtered_data

In [ ]:
plt.plot(filtered_data.pressure, filtered_data.ctd_depth)

In [ ]:
sliced_data['ox_downcast'] = sliced_data.ox_downcast.where(np.abs(sliced_data.temp - ds2.temp) < 0.1)

ds2['o2'] = ds2.o2.where(np.abs(sliced_data.temp - ds2.temp) < 0.1)

#oxygendown_station_15['ox_downcast'] = oxygendown_station_15.ox_downcast.where(oxygendown_station_15.temp < 18.0)
#filtered_data['o2'] = filtered_data.o2.where(np.abs(oxygendown_station_15.temp - filtered_data.temp) < 0.1)



In [ ]:
plt.scatter(ds2.o2,sliced_data.ox_downcast.values)
plt.xlabel('ox glider')
plt.ylabel('ctd ox')
plt.plot([140,210],[140,210])
#plt.plot(sliced_data.pressure.values,sliced_data.ox_downcast.values)

#plt.plot(ds2.pressure.values,ds2.o2)

#plt.gca().invert_yaxis()

In [ ]:
offset = (sliced_data.ox_downcast - ds2.o2).mean()
offset

In [ ]:
ds["o2"]=ds.o2+offset 

ds

In [ ]:
# Drop the unwanted variables from the dataset
filtered_data = ds.drop_vars(['cast', 'station', 'max_pressure', 'depth'])
filtered_data

In [ ]:
plt.figure(figsize=(12, 6))


plt.pcolormesh(filtered_data.dives, filtered_data.ctd_depth, filtered_data.o2)


#plt.figure(figsize=(12, 6))

# Add colorbar
plt.colorbar(label='Dissolved Oxygen (micromoles/kg)')


# Add labels and title
plt.xlabel('Dives', fontsize=12)
plt.ylabel('Depth (m)', fontsize=12)

# Invert y-axis
plt.gca().invert_yaxis()
#plt.plot()


In [ ]:

plt.plot(oxygendown_station_15.pressure.values,oxygendown_station_15.ox_downcast.values)
plt.xlim(0,1000)

plt.plot(filtered_data.pressure.values,filtered_data.o2.sel(dives=9.5).values)
#plt.legend()

plt.gca().invert_yaxis()
plt.plot()


In [ ]:
filtered_data.to_netcdf('ds_calibrated.nc')